In [1]:
import $ivy.`org.apache.spark::spark-sql:2.1.0` 
import $ivy.`InitialDLab:simba_2.11:1.0`
import $ivy.`org.slf4j:slf4j-jdk14:1.7.25`


Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-core/1.14.0/jts-core-1.14.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-core/1.14.0/jts-core-1.14.0.pom
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.11/3.0.1/scalactic_2.11-3.0.1.pom
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.11/3.0.1/scalactic_2.11-3.0.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-core/1.14.0/
Downloaded https://repo1.maven.org/maven2/org/scalactic/scalactic_2.11/3.0.1/
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-parent/1.14.0/jts-parent-1.14.0.pom
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-parent/1.14.0/jts-parent-1.14.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-parent/1.14.0/
Downloaded https://repo1.maven.org/maven2/com/vividsolutions/jts-core/1.14.0/jts-core-1.14.0.jar.sha1
Downloaded https://repo1.maven.org/maven2/org/s

import $ivy.$                                   

import $ivy.$                           

import $ivy.$                             


In [2]:
import org.slf4j.{Logger, LoggerFactory}
import org.apache.spark.sql.simba.{Dataset, SimbaSession}
import org.apache.spark.sql.simba.index.RTreeType
import org.apache.spark.sql.types.{StructType, StructField, LongType, DoubleType, IntegerType}

import org.apache.spark.sql.functions._
import scala.collection.JavaConverters._
import scala.collection.mutable.ListBuffer


import org.slf4j.{Logger, LoggerFactory}

import org.apache.spark.sql.simba.{Dataset, SimbaSession}

import org.apache.spark.sql.simba.index.RTreeType

import org.apache.spark.sql.types.{StructType, StructField, LongType, DoubleType, IntegerType}


import org.apache.spark.sql.functions._

import scala.collection.JavaConverters._

import scala.collection.mutable.ListBuffer


In [4]:
// spark://169.235.27.134:7077
val simba = SimbaSession.builder()
      .master("local[*]")
      .appName("MaximalDistanceFinder")
      .config("simba.index.partitions", 1024)
      .config("spark.cores.max", 32)
      .getOrCreate()


simba: SimbaSession = org.apache.spark.sql.simba.SimbaSession@681de89c

In [5]:
val logger: Logger = LoggerFactory.getLogger("myLogger")
var timer: Long = 0L

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this)
case class ST_Point(id: Long, x: Double, y: Double, t: Int = -1)
case class Flock(start: Int, end: Int, ids: String, x: Double = 0.0, y: Double = 0.0)


logger: Logger = org.slf4j.impl.Log4jLoggerAdapter(myLogger)
timer: Long = 0L
defined class ST_Point
defined class Flock

In [6]:
import simba.implicits._
import simba.simbaImplicits._


import simba.implicits._

import simba.simbaImplicits._


In [7]:
val id = StructField("id", LongType)
val x = StructField("x", DoubleType)
val y = StructField("y", DoubleType)
val t = StructField("t", IntegerType)
val point_schema = StructType(Array(id, x, y, t))


id: StructField = StructField(id,LongType,true)
x: StructField = StructField(x,DoubleType,true)
y: StructField = StructField(y,DoubleType,true)
t: StructField = StructField(t,IntegerType,true)
point_schema: StructType = StructType(
  StructField(id,LongType,true),
  StructField(x,DoubleType,true),
  StructField(y,DoubleType,true),
  StructField(t,IntegerType,true)
)

In [10]:
val research_home = scala.util.Properties.envOrElse("RESEARCH_HOME", "/home/and/Documents/PhD/Research/" )
val filename = s"${research_home}Datasets/Berlin/berlin0-2.tsv"

val pointset = simba.read
      .option("header", "false")
      .option("sep", "\t")
      .schema(point_schema)
      .csv(filename)
      .as[ST_Point]
      .cache()
val nPointset = pointset.count()


research_home: String = "/home/and/Documents/PhD/Research/"
filename: String = "/home/and/Documents/PhD/Research/Datasets/Berlin/berlin0-2.tsv"
pointset: org.apache.spark.sql.Dataset[ST_Point] = [id: bigint, x: double ... 2 more fields]
nPointset: Long = 54732L

In [ ]:
val epsilon = 50
val sample = pointset.filter(_.t == 0).sample(false, 0.5, 42).cache()
sample.orderBy("id").show(truncate =false)
val p1 = sample.select("id", "x", "y").toDF("id1", "x1", "y1")
p1.index(RTreeType, "rtP1", Array("x1", "y1"))
val p2 = sample.select("id", "x", "y").toDF("id2", "x2", "y2")
p2.index(RTreeType, "rtP2", Array("x2", "y2"))
val join = p1.distanceJoin(p2, Array("x1", "y1"), Array("x2", "y2"), epsilon).filter(j => j.getLong(0) < j.getLong(3)).cache()
val nJoin = join.count()
join.orderBy("id1", "id2").show(truncate = false)


+---+---------+---------+---+
|id |x        |y        |t  |
+---+---------+---------+---+
|3  |18071.8  |21890.985|0  |
|4  |16582.549|23500.546|0  |
|6  |36818.069|26567.175|0  |
|7  |26567.764|23949.332|0  |
|8  |23093.249|23491.667|0  |
|9  |14865.52 |17079.365|0  |
|11 |23333.804|27343.981|0  |
|12 |23122.741|19933.776|0  |
|14 |19227.697|22639.845|0  |
|15 |27498.866|14628.038|0  |
|17 |23957.575|18650.487|0  |
|19 |20353.159|26565.008|0  |
|23 |21769.334|18149.96 |0  |
|24 |31699.043|14230.055|0  |
|27 |32280.197|19959.179|0  |
|30 |19436.481|21842.427|0  |
|31 |30495.447|22703.329|0  |
|33 |36880.811|13892.571|0  |
|35 |22137.34 |20929.995|0  |
|37 |23834.526|18348.774|0  |
+---+---------+---------+---+
only showing top 20 rows

+---+---------+---------+-----+---------+---------+
|id1|x1       |y1       |id2  |x2       |y2       |
+---+---------+---------+-----+---------+---------+
|4  |16582.549|23500.546|7705 |16572.58 |23501.818|
|6  |36818.069|26567.175|15319|36820.311|26573

In [ ]:
simba.close()